<a href="https://colab.research.google.com/github/Uzair-DeVops/Quater-2/blob/main/Project_02_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma

In [120]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud


#1 . **FIIRST** YOU NEED TO DOWNLOAD DESRIED PDF FILE AND UPLOAD IT TO COLAB
HERE IS THE FILE LINK : https://drive.google.com/file/d/1D85ntKHxp6zhJerqFbGe4OZh_NkpZRBb/view?usp=sharing


# 2 . Loading Pdf

In [121]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = loader.load()


In [122]:
len(data)

11

# 3 . Spliting text

In [123]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  43


# 4 . Using API key

In [124]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_3')

# 5 . HERE YOU NEED TO DOWNLOAD ANOTHER FILE AND UPLOAD IT TO COLBA :
HERE IS THE LINK : https://drive.google.com/file/d/1afKObnhCZ9UeXyMuqiV2t4BJ9Nkvgm4D/view?usp=sharing

# 6 . Using json file

In [125]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/test.json"


In [126]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

# 8 . Creating embedding model

In [127]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)

# 9 . Embedding data and Retriver

In [128]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


# 10 . Creating LLM

In [129]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.3 , max_tokens=1000)

# 11 . Defining user_prompt

In [130]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 12 . Creating chains

In [131]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 12 . Asking question which is not in the pdf




In [132]:
response = rag_chain.invoke({"input" : "what is uzair"})
print(response["answer"])

I'm sorry, but the provided context does not contain any information about "uzair". Therefore, I cannot answer your question.



# 13 . Asking question which is  in the pdf

In [133]:
response = rag_chain.invoke({"input" : "what is multi attention"})
print(response["answer"])

Multi-head attention is calculated by projecting the queries, keys, and values into multiple lower-dimensional spaces, performing attention in each space, and then concatenating and projecting the results. The formula is MultiHead(Q,K,V ) = Concat(head1,..., headh)WO where headi = Attention(QWQ
i ,KW K
i ,VW V
i). In this model, 8 parallel attention layers, or heads, are used.

